# Populando o Pinecone com Base de Conhecimento
Indexação de informações turísticas de Rio de Janeiro e Paris para uso no RAG.

In [ ]:
import numpy as np
from pinecone import Pinecone
pinecone_client = Pinecone()
indice = pinecone_client.Index('turismo')

In [ ]:
# Carregar base de conhecimento
with open('base_conhecimento_rio_paris.txt', 'r', encoding='utf-8') as f:
    linhas = f.readlines()
docs = [linha.strip() for linha in linhas if linha.strip() and not linha.startswith('#')]
ids = [f'doc_{i}' for i in range(len(docs))]
# Vetorização simples (exemplo):
vetores = [np.random.normal(0, 1, 2048).tolist() for _ in docs]
indice.upsert(vectors=list(zip(ids, vetores)))